In [21]:
import numpy as np
import xtrack as xt         
import xpart as xp
import xdeps as xd   
import yaml
import os
import inspect
import pickle
import scipy

In [9]:
!pwd


/afs/cern.ch/work/c/cmaccani/xsuite_sim/twocryst_sim/input_files/phase_scan


In [10]:
with open('./knobs_db/knobs_159.pkl', 'rb') as f:
    knobs = pickle.load(f)

In [11]:
with open('../HL_IR7_phase_advance/knob.pkl', 'rb') as f:
    knobs1 = pickle.load(f)

In [12]:
knobs == knobs1

True

In [42]:
beam          = 2   #run_dict['beam']
plane         = "V" #run_dict['plane']
TCCS_name     = 'tccs.5r3.b2'
TCCP_name     = 'tccp.4l3.b2'
TARGET_name   = 'target.4l3.b2'
TCLA_name     = 'tcla.a5l3.b2'
TCP_name      = f"tcp.{'c' if plane=='H' else 'd'}6{'l' if beam=='1' else 'r'}7.b{beam}"

def load_line(line_name, coll_file, particle_ref):
    if coll_file.endswith('.yaml'):
        with open(coll_file, 'r') as stream:
            coll_dict = yaml.safe_load(stream)['collimators'][f'b2']

    # Load from json
    line = xt.Line.from_json(line_name)
    line.particle_ref = particle_ref

    end_s = line.get_length()

    TCCS_loc = end_s - 6773.7 #6775
    TCCP_loc = end_s - 6653.3 #6655
    TARGET_loc = end_s - (6653.3 + coll_dict[TCCP_name]["length"]/2 + coll_dict[TARGET_name]["length"]/2)

    line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name='tccs.5r3.b2')
    line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccs.5r3.b2_aper')
    line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name='tccp.4l3.b2')
    line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccp.4l3.b2_aper')
    line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name='target.4l3.b2')
    line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='target.4l3.b2_aper')




    return line

In [45]:
coll = os.path.expandvars('${HOME_TWOCRYST}/input_files/colldbs/HL_tight_twocryst.yaml')
line = os.path.expandvars("${HOME_TWOCRYST}/input_files/phase_scan/b4_sequence_patched_phadv_150.json")
particle_ref_HL= xp.Particles(p0c=7000e9, q0=1, mass0=xp.PROTON_MASS_EV)

In [46]:
tw = load_line(line, coll, particle_ref_HL).twiss(method='4d').to_pandas()

Loading line from dict:   0%|          | 0/151124 [00:00<?, ?it/s]

Done loading line from dict.           
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [47]:
def calculate_delta_phase_adv(tw):
    return (float(tw[tw.name == TCCS_name]['muy'])% 1* 2*np.pi - float(tw[tw.name == TCP_name]['muy'])% 1* 2*np.pi)*180/np.pi

In [48]:
calculate_delta_phase_adv(tw)

150.46979031963522